# Bit-Rate Control in InterCom

Maximilian Hebeis, Artur Jakubowski, Weronika Niezorawska, Julia Plichta

### 2.1 Which data-ordering performs better?

### 2.2 Estimate the bit-rate in an Internet link

As mentioned on the web page, there are two possible ways to measure the bitrate: Using a dedicated tool such as _iPerf_ or estimating the link throughput using _ping_. We tried both to get an estimation. The two machines between which we set up a connection where located in Bamberg, Germany, and Jena, Germany, approximately 126 km from each other. The server was stationed in Bamberg, while the client was located in Jena.
![title](imgs/bitrate01.png)
Source: https://www.mapdevelopers.com/distance_from_to.php

First we set up _iPerf_ on both hosts:
![title](imgs/bitrate02.png)
![title](imgs/bitrate06.png)

According to _iPerf_, a bitrate of between 8.36 and 8.48 Mbit/s could be established between the host and the server.

Subsequently, we used _ping_ to get an alternative measurement of the bitrate between the two hosts. This time, the host in Jena was pinged by the host in Bamberg. We ran _ping_ using the smallest possible payload that provides RTTs values, which turned out to be 16 bytes (cf. https://serverfault.com/questions/361147/why-doesnt-ping-show-rtt) and using the largest possible payload, which we determined to be 1250 bytes by trial-and-error.

![title](imgs/bitrate03.png)
![title](imgs/bitrate04.png)

Using the formula from the lecture to determine $ t_p $ with the value for $ RTT_{min} $ we got from pinging with the smallest possible payload:
$$ t_p \approx \frac{RTT_{min}}{2} = \frac{0.054337 s}{2} = 0.0271685 s $$
Subsequently, we can compute $ t_t $ using the $ RTT_{max} $ we got from pinging with the largest possible payload:
$$ t_t = \frac{RTT_{max} - 2t_p}{2} = \frac{0.067597 s - 2 \cdot 0.0271685 s}{2} = 0.00663 s $$
Finally, we determined the bitrate _b_:
$$ b = \frac{B}{t_t} = \frac{1250 \cdot 8 bits}{0.00663 s} \approx 1.508 Mbit/s $$

### 2.3 Simulate the link, 2.4 Compute RD (Rate/Distortion) curves

To simulate different links and compute the corresponding RMSE for each of the four bitrate control implementations, we implemented a new module called `RMSE_chunks.py`. This module is supposed to compare the sent and received chunks in the use case of using a single InterCom instance to transfer an audio file over a (simulated) localhost link. The base class `RMSEChunks_no` inherits from `BR_Control_No` and is thus used to measure the RMSEs for this implementation, the three other classes `RMSEChunks_add_lost`, `RMSE_Chunks_lost` and `RMSEChunks_conservative` are used to measure the respective RMSEs for the other implementations.

In [ ]:
#!/usr/bin/env python
# PYTHON_ARGCOMPLETE_OK

'''An Intercom wrapper employing different implementations of bit-rate control 
and computing the RMSE over the sent and received chunks (if sent and received 
in a single InterCom instance).'''

import minimal
import buffer
import logging
import numpy as np
import matplotlib.pyplot as plt
import os
import signal
import BR_control_no
import BR_control_add_lost
import BR_control_lost
import BR_control_conservative

minimal.parser.add_argument("-z", "--bitrate_imp", type=int, default=0, help="Select bitrate control implementation")

class RMSEChunks_no(BR_control_no.BR_Control_No):
    def __init__(self):
        logging.disable(logging.CRITICAL)
        super().__init__()

        self.sent_chunks = np.empty((1024, 2))
        self.recd_chunks = np.empty((1024, 2))
    
    def _read_io_and_play(self, outdata, frames, time, status):
        read_chunk = super()._read_io_and_play(outdata, frames, time, status)
        self.sent_chunks = np.append(self.sent_chunks, read_chunk)
        return read_chunk
    
    def receive_and_buffer(self):
        chunk_number = super().receive_and_buffer()
        self.recd_chunks = np.append(self.recd_chunks, self._buffer[chunk_number % self.cells_in_buffer])
        return chunk_number
    
    def read_chunk_from_file(self):
        chunk = self.wavfile.buffer_read(minimal.args.frames_per_chunk, dtype='int16')
        #print(len(chunk), args.frames_per_chunk)
        if len(chunk) < minimal.args.frames_per_chunk*4:
            logging.warning("Input exhausted! :-/")
            #print(self.recd_chunks.dtype)
            #print(self.sent_chunks.dtype)
            #print(self.recd_chunks)

            rmse = np.sqrt(np.mean(np.square(np.array(self.recd_chunks, dtype='int16') - np.array(self.sent_chunks, dtype='int16'))))
            print(rmse)
            pid = os.getpid()
            os.kill(pid, signal.SIGINT)
            return self.zero_chunk
        chunk = np.frombuffer(chunk, dtype=np.int16)
        #try:
        chunk = np.reshape(chunk, (minimal.args.frames_per_chunk, self.NUMBER_OF_CHANNELS))
        #except ValueError:
            #logging.warning("Input exhausted! :-/")
            #pid = os.getpid()
            #os.kill(pid, signal.SIGINT)
            #self.input_exhausted = True
        return chunk

class RMSEChunks_add_lost(RMSEChunks_no, BR_control_add_lost.BR_Control_Add_Lost):
    pass

class RMSEChunks_lost(RMSEChunks_no, BR_control_lost.BR_Control_Lost):
    pass

class RMSEChunks_conservative(RMSEChunks_no, BR_control_conservative.BR_Control_Conservative):
    pass

np.seterr(divide='ignore', invalid='ignore')

try:
    import argcomplete  # <tab> completion for argparse.
except ImportError:
    logging.warning("Unable to import argcomplete (optional)")

if __name__ == "__main__":
    minimal.parser.description = __doc__

    try:
        argcomplete.autocomplete(minimal.parser)
    except Exception:
        logging.warning("argcomplete not working :-/")

    minimal.args = minimal.parser.parse_known_args()[0]

    if minimal.args.list_devices:
        print("Available devices:")
        print(sd.query_devices())
        quit()

    intercom = RMSEChunks_no()
    
    match minimal.args.bitrate_imp:
        case 0:
            pass
        case 1:
            intercom = RMSEChunks_add_lost()
        case 2:
            intercom = RMSEChunks_lost()
        case 3:
            intercom = RMSEChunks_conservative()
    
    try:
        intercom.run()
    except KeyboardInterrupt:
        #minimal.parser.exit("\nSIGINT received")
        minimal.parser.exit()
    finally:
        intercom.print_final_averages()

Logging and the exit message for the received _SIGINT_ were disabled, as `RMSE_chunks.py` had to be called from the outside for every simulated bitrate -- the RMSE values for each single bitrate could be most easily collected if they were the only output over `stdout`. To simulate the bitrates and collect the RMSEs for each implementation-bitrate pair automatically we implemented a small Python script called `plot_rd_curve.py`, calling `RMSE_chunks.py` (with the flag -O, as this sets `__debug__ = false` and thus disables the spinner) and `tc` for each combination of implementation and bitrate and collecting each RMSE value in a two-dimensional array. In the end, we receive four `matplotlib` plots, one for each of the implementations.

In [ ]:
#!/usr/bin/env python
# PYTHON_ARGCOMPLETE_OK

import subprocess
import matplotlib.pyplot as plt

class PlotRDCurve:
    def __init__(self, bitrates = list(range(100, 1300, 100))):
        self.bitrates = bitrates
        self.imps = {   0: 'BR_control_no.py',
                        1: 'BR_control_add_lost.py',
                        2: 'BR_control_lost.py',
                        3: 'BR_control_conservative.py'}
        self.imp_rmses = []
    
    def plot(self):
        for imp_no in range(0, 4):
            rmse_arr = []
            for br in self.bitrates:
                self.simulate_link(br)
                rmse = subprocess.run(['python', '-O', 'RMSE_chunks.py', '-f', 'nuclear.wav', '-z', str(imp_no)], stdout=subprocess.PIPE).stdout
                print(rmse)
                rmse = float(rmse.decode('utf-8').rstrip())
                rmse_arr.append(rmse)
                self.delete_sim_rule(br)
            self.imp_rmses.append(rmse_arr)
        
            plt.plot(self.bitrates, rmse_arr, marker='o')
            plt.xlabel('Bitrate (kbps)')
            plt.ylabel('RMSE')
            plt.title('R/D curve for ' + self.imps[imp_no])
            plt.savefig(self.imps[imp_no]+'.png')
            plt.clf()
    
    def simulate_link(self, bitrate):
        subprocess.run(['tc', 'qdisc', 'add', 'dev', 'lo', 'root', 'handle', '1:', 'tbf', 'rate', str(bitrate)+'kbit', 'burst', '32kbit', 'limit', '32kbit'])
        #subprocess.run(['tc', 'qdisc', 'add', 'dev', 'lo', 'parent', '1:1', 'handle', '10:', 'netem', 'delay', '100ms', '10ms', '25%', 'distribution', 'normal'])
    
    def delete_sim_rule(self, bitrate):
        #for bitrate in self.bitrates:
            #subprocess.run(['tc', 'qdisc', 'delete', 'dev', 'lo', 'parent', '1:1', 'handle', '10:', 'netem', 'delay', '100ms', '10ms', '25%', 'distribution', 'normal'])
        subprocess.run(['tc', 'qdisc', 'delete', 'dev', 'lo', 'root', 'handle', '1:', 'tbf', 'rate', str(bitrate)+'kbit', 'burst', '32kbit', 'limit', '32kbit'])

if __name__ == "__main__":
    plotter = PlotRDCurve()
    plotter.plot()